In [24]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
df = pd.read_csv ('../Great_Streets_Pedestrian_Survey_2015.csv')

In [2]:
columns = ['ID',
'Surveyed Date',                                                                                            
'Surveyed Time',                                                                                             
'Surveyed End Date',                                                                                        
'What brings you to this neighborhood today?',                                                           
'How often do you visit the neighborhood?',                                                              
'How much time do you typically spend when you visit?',                                                  
'How do you typically get to this neighborhood? Select Primary (next: secondary)',                       
'How do you typically get to this neighborhood? (select secondary)',
'How much money do you typically spend when you visit? (at businesses in the area)',                     
'In this neighborhood, how often do you recognize someone by face or name?',                             
'This neighborhood is clean and well-maintained?',                                                        
'This neighborhood is safe?',                                                                             
'This neighborhood is attractive?',                                                                      
'This neighborhood is active and lively?',                                                              
'This neighborhood is a place where its easy to talk to others? (who I may not have known before)',     
'This neighborhood has a strong identity?',                                                              
'This neighborhood is in an area with a lot of noise?',                                                  
'When you visit the neighborhood, how often are you accompanied by anyone under 16 years old?',         
'When you visit the neighborhood, how often are you accompanied by anyone over 65 years old?',          
'When you visit the neighborhood, how often are you accompanied by anyone mobility assisted?',          
'When you visit the neighborhood, how often are you accompanied by anyone who is a family member?',     
'What year were you born?',                                                                            
'Your gender identity?',                                                                                
'Your ethnic identity?',                                                                                
'Your racial identity?',                                                                                
'What is your home zipcode?',
'Is it raining?',
'Location', 
'Location Between',
'Location Between AND',
'Side of street for this interview']

df.columns = columns

In [3]:
new_cols = ['I live Here', 
 'I work Here', 
 'Passing through en route to somewhere else', 
 'Clinic', 
 'To meet friends',
 'To shop',
 'Dentist',
 'Joining brother at the dentist',
 'For music / art',
 'Going to/from school',
 'To dine / drink',
 'To meet friends',
 'Attractions',
 'Fun',
 'Fresh air', 
 'Volunteer',
]


In [4]:
df.head()

,ID,Surveyed Date,Surveyed Time,Surveyed End Date,What brings you to this neighborhood today?,How often do you visit the neighborhood?,How much time do you typically spend when you visit?,How do you typically get to this neighborhood? Select Primary (next: secondary),How do you typically get to this neighborhood? (select secondary),How much money do you typically spend when you visit? (at businesses in the area),"In this neighborhood, how often do you recognize someone by face or name?",This neighborhood is clean and well-maintained?,This neighborhood is safe?,This neighborhood is attractive?,This neighborhood is active and lively?,This neighborhood is a place where its easy to talk to others? (who I may not have known before),This neighborhood has a strong identity?,This neighborhood is in an area with a lot of noise?,"When you visit the neighborhood, how often are you accompanied by anyone under 16 years old?","When you visit the neighborhood, how often are you accompanied by anyone over 65 years old?","When you visit the neighborhood, how often are you accompanied by anyone mobility assisted?","When you visit the neighborhood, how often are you accompanied by anyone who is a family member?",What year were you born?,Your gender identity?,Your ethnic identity?,Your racial identity?,What is your home zipcode?,Is it raining?,Location,Location Between,Location Between AND,Side of street for this interview
0,9328599,05/26/2015,17:00:42,05/26/2015 05:05:00 PM,I Live Here,Once a week,< 10 minutes,On foot,Car,$5-10,NaN,Don't Know / No Opinion,NaN,NaN,NaN,NaN,NaN,NaN,Often,Never,Sometimes,Sometimes,1975.0,Female,Hispanic or Latino,Native Hawaiian or Pac Islander,90004.0,NaN,Venice Blvd,NaN,NaN,North
1,9328602,05/26/2015,17:00:43,05/26/2015 05:05:00 PM,"I Live Here, I Work Here",Daily,NaN,Car,On foot,$5-10,Often,Moderately Disagree,Moderately Disagree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Van Nuys,NaN,NaN,South
2,9353258,05/27/2015,7:32:49,05/27/2015 07:37:00 AM,"I Live Here, Passing through en route to somew...",Several times a week,10-30 minutes,On foot,On foot,None,Often,Moderately Disagree,Moderately Disagree,Moderately Disagree,Don't Know / No Opinion,NaN,NaN,Strongly Agree,Never,Never,Never,Never,NaN,Female,Hispanic or Latino,White,91331.0,No,Van Nuys,Laurel Canyon,San Fernando Rd.,East
3,9353259,05/27/2015,8:05:18,05/27/2015 08:08:00 AM,I Live Here,Daily,< 10 minutes,Car,Car,$5-10,Often,Strongly Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Strongly Agree,Often,Never,Never,Often,NaN,Female,Hispanic or Latino,White,93133.0,No,Van Nuys,NaN,NaN,East
4,9353261,05/27/2015,8:09:08,05/27/2015 08:15:00 AM,Clinic,Once a month,10-30 minutes,Car,Car,None,Never,NaN,Moderately Agree,NaN,NaN,NaN,NaN,NaN,Never,Never,Never,Sometimes,1938.0,Male,Hispanic or Latino,White,91331.0,No,Van Nuys,NaN,NaN,East


In [5]:
new_cols = ['I Live Here', 
 'I Work Here', 
 'Passing through en route to somewhere else', 
 'Clinic', 
 'To meet friends',
 'To shop',
 'Dentist',
 'Joining brother at the dentist',
 'For music / art',
 'Going to/from school',
 'To dine / drink',
 'Attractions',
 'Fun',
 'Fresh air', 
 'Volunteer',
]

In [6]:
import re
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def categorise(row, category):
    category_no_punct = re.sub(r'[^\w\s]', '', category)
    question_no_punct = re.sub(r'[^\w\s]', '', row['What brings you to this neighborhood today?'])

    category_words = category_no_punct.lower().split()
    question_words = question_no_punct.lower().split()
    result=[word for word in category_words if word in question_words]
    
    if similar(category_words, result) == 1.0 or similar(category_words, result) >= 9.0:
        return True
    else:
        return False

In [7]:
for category in new_cols:
    df[category] = df.apply(lambda row: categorise(row, category), axis=1)

In [18]:
import requests
import json
import urllib
base_url= "https://maps.googleapis.com/maps/api/geocode/json?"
AUTH_KEY = ""



In [16]:
def geo_street(street_01, street_02):
    parameters = {"address": street_01+" and " + street_02 +", Los Angeles",
              "key": AUTH_KEY}

    r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
    
    data = json.loads(r.content)
    if data['status'] == 'OK' and  data['results'][0]['geometry']['location_type'] == 'GEOMETRIC_CENTER':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        formatted_address = data['results'][0]['address_components'][0]['long_name']
        geo_type = data['results'][0]['geometry']['location_type']
        zipcode = data['results'][0]['address_components'][6]['long_name']
        area = data['results'][0]['address_components'][1]['long_name']
    elif  data['status'] == 'OK' and  data['results'][0]['geometry']['location_type'] == 'APPROXIMATE':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        formatted_address = data['results'][0]['address_components'][0]['long_name']
        geo_type = data['results'][0]['geometry']['location_type']
        zipcode = np.nan
        area = data['results'][0]['address_components'][1]['long_name']
    elif data['status'] == 'ZERO_RESULTS':
        lat = np.nan
        lng = np.nan
        formatted_address = np.nan
        geo_type = np.nan
        zipcode = np.nan
        area = np.nan
        
    return lat, lng,formatted_address, zipcode, geo_type, area
    

In [17]:
v,w,x,y,z,zz = geo_street('sunset', 'vine')

print(v,w,x,y,z,zz)

34.0980023 -118.3266816 Sunset Boulevard & Vine Street 90028 GEOMETRIC_CENTER Central LA


In [20]:
#cannot change float column of values with nan into string
df['What is your home zipcode?'].fillna(0, inplace=True) #change nan values to 0
df['What is your home zipcode?'] = df['What is your home zipcode?'].apply(int) #change zipcode float values to int
df['What is your home zipcode?']= df['What is your home zipcode?'].apply(str) #change zipcode str 
df['What is your home zipcode?'] = df['What is your home zipcode?'].replace('0', 'none')

In [22]:
#cannot change float column of values with nan into string
df['What year were you born?'].fillna(0, inplace=True) #change nan values to 0
df['What year were you born?'] = df['What year were you born?'].apply(int)  
df['What year were you born?']= df['What year were you born?'].apply(str)  
df['What year were you born?'] = df['What year were you born?'].replace('0', 'none')

In [23]:
df

,ID,Surveyed Date,Surveyed Time,Surveyed End Date,What brings you to this neighborhood today?,How often do you visit the neighborhood?,How much time do you typically spend when you visit?,How do you typically get to this neighborhood? Select Primary (next: secondary),How do you typically get to this neighborhood? (select secondary),How much money do you typically spend when you visit? (at businesses in the area),"In this neighborhood, how often do you recognize someone by face or name?",This neighborhood is clean and well-maintained?,This neighborhood is safe?,This neighborhood is attractive?,This neighborhood is active and lively?,This neighborhood is a place where its easy to talk to others? (who I may not have known before),This neighborhood has a strong identity?,This neighborhood is in an area with a lot of noise?,"When you visit the neighborhood, how often are you accompanied by anyone under 16 years old?","When you visit the neighborhood, how often are you accompanied by anyone over 65 years old?","When you visit the neighborhood, how often are you accompanied by anyone mobility assisted?","When you visit the neighborhood, how often are you accompanied by anyone who is a family member?",What year were you born?,Your gender identity?,Your ethnic identity?,Your racial identity?,What is your home zipcode?,Is it raining?,Location,Location Between,Location Between AND,Side of street for this interview,I Live Here,I Work Here,Passing through en route to somewhere else,Clinic,To meet friends,To shop,Dentist,Joining brother at the dentist,For music / art,Going to/from school,To dine / drink,Attractions,Fun,Fresh air,Volunteer
0,9328599,05/26/2015,17:00:42,05/26/2015 05:05:00 PM,I Live Here,Once a week,< 10 minutes,On foot,Car,$5-10,NaN,Don't Know / No Opinion,NaN,NaN,NaN,NaN,NaN,NaN,Often,Never,Sometimes,Sometimes,1975,Female,Hispanic or Latino,Native Hawaiian or Pac Islander,90004,NaN,Venice Blvd,NaN,NaN,North,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,9328602,05/26/2015,17:00:43,05/26/2015 05:05:00 PM,"I Live Here, I Work Here",Daily,NaN,Car,On foot,$5-10,Often,Moderately Disagree,Moderately Disagree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,NaN,NaN,NaN,none,NaN,Van Nuys,NaN,NaN,South,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,9353258,05/27/2015,7:32:49,05/27/2015 07:37:00 AM,"I Live Here, Passing through en route to somew...",Several times a week,10-30 minutes,On foot,On foot,None,Often,Moderately Disagree,Moderately Disagree,Moderately Disagree,Don't Know / No Opinion,NaN,NaN,Strongly Agree,Never,Never,Never,Never,none,Female,Hispanic or Latino,White,91331,No,Van Nuys,Laurel Canyon,San Fernando Rd.,East,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,9353259,05/27/2015,8:05:18,05/27/2015 08:08:00 AM,I Live Here,Daily,< 10 minutes,Car,Car,$5-10,Often,Strongly Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Moderately Disagree,Strongly Agree,Often,Never,Never,Often,none,Female,Hispanic or Latino,White,93133,No,Van Nuys,NaN,NaN,East,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,9353261,05/27/2015,8:09:08,05/27/2015 08:15:00 AM,Clinic,Once a month,10-30 minutes,Car,Car,None,Never,NaN,Moderately Agree,NaN,NaN,NaN,NaN,NaN,Never,Never,Never,Sometimes,1938,Male,Hispanic or Latino,White,91331,No,Van Nuys,NaN,NaN,East,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,11202231,06/06/2015,17:09:27,06/06/2015 05:24:00 PM,I Live Here,Daily,NaN,Bike,On foot,$5-10,Sometimes,Strongly Disagree,Moderately Agree,Moderately Disagree,Moderately Agree,Moderately Agree,NaN,Strongly Agree,Sometimes,S